# Image Classification with PyTorch
Pytorch has been both researcher's and engineer's preferred choice of framework for DL development but when it comes to productionizing pytorch models, there still hasn't been a consensus on what to use. This guide run you through building a simple image classification model using Pytorch and then deploying that to RedisAI. Let's start with importing the necessary packages

In [1]:
import torchvision.models as models
import torch

import json
import time
from redisai import Client
import ml2rt
from skimage import io

import os
from redisai import Client

## Build Model
For this example, we use a pretrained model from torchvision for image classification - the renowned resnet50. Since RedisAI is a C/C++ runtime, we'd need to export the torch model into [TorchScript](https://pytorch.org/docs/stable/jit.html). Here is how to do it but you can read more about TorchScript in the attached link

In [2]:
model = models.resnet50(pretrained=True)
model.eval()

scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, 'resnet50.pt')

## Setup RedisAI
This tutorial assumes you already have a RedisAI server running. The easiest way to setup one instance is using docker

```
docker run -p 6379:6379 redislabs/redisai:latest-cpu-x64-bionic
```

Take a look at this [quickstart](https://oss.redis.com/redisai/quickstart/) for more details. Here we setup the connection credentials and ping the server to verify we can talk 

In [3]:
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = int(os.getenv("REDIS_PORT", 6379))
con = Client(host=REDIS_HOST, port=REDIS_PORT)
con.ping()

True

## Load model
Next step is to load the model we trained above into RedisAI for serving. We are using a convinent package [ml2rt](https://pypi.org/project/ml2rt/) here for loading but it's not a mandatory dependency if you want to keep your `requirements.txt` small. Take a look at the `load_model` function. This will give us a binary blob of the model we have built above. We need to send this to RedisAI and also inform which backend we'd like to use and which device this should run on. We'll set the model on a key so we can reference this key later

Note: If you want to run on GPU, take a look at the above quick start to setup RedisAI on GPU

In [4]:
model = ml2rt.load_model("resnet50.pt")
con.modelstore("pytorch_model", backend="TORCH", device="CPU", data=model)

'OK'

## Load script
Why do you need Script? It's very likely that your deep learning model would have a pre/post processing step, like changing the dimensionality of the input (adding batch dimension) or doing normalizatoin etc. You normally do this from your client code and send the processed data to model server. With script, you can club this into your model serving pipeline. Script is one of the powerful feature of RedisAI. RedisAI Scripts are built on top of [TorchScript](https://pytorch.org/docs/stable/jit.html) and it's recommended to take a look if TorcScript is new to you. Torchscript is a subset of python programming langauge i.e it looks and smells like python but all the python functionalities are not available in torchscript. Now if you are wondering what's the benefit of TorchScript in RedisAI, there are few

- It runs on a highly effecient C++ runtime
- It can pipeline your preprocessing and postprocessing jobs, right where your model and data resides. So no back and forth of huge data blobs between your model server and pre/post processing scripts
- It can run in a single redis pipeline or in RedisAI Dag which makes serving channel implementation smooth
- You can use it with any framework, not just pytorch

You can load the script from a file (`ml2rt.load_script` does this for you) which is probably your workflow normally since you save the script in a file but here we pass the string into the `scriptstore` method

In [5]:
script = """
def pre_process(tensors: List[Tensor], keys: List[str], args: List[str]):
    image = tensors[0]
    mean = torch.zeros(3).float().to(image.device)
    std = torch.zeros(3).float().to(image.device)
    mean[0], mean[1], mean[2] = 0.485, 0.456, 0.406
    std[0], std[1], std[2] = 0.229, 0.224, 0.225
    mean = mean.unsqueeze(1).unsqueeze(1)
    std = std.unsqueeze(1).unsqueeze(1)
    temp = image.float().div(255).permute(2, 0, 1)
    return temp.sub(mean).div(std).unsqueeze(0)


def post_process(tensors: List[Tensor], keys: List[str], args: List[str]):
    output = tensors[0]
    return output.max(1)[1]
"""
con.scriptstore("processing_script", device="CPU", script=script, entry_points=("pre_process", "post_process"))

'OK'

## Load the image and final classes
Here we load the input image and the final classes to find the predicted output

In [6]:
image = io.imread("data/cat.jpg")
class_idx = json.load(open("data/imagenet_classes.json"))

## Run the model serving pipeline
Here we run the serving pipeline one by one and finally fetch the results out. The pipeline is organized into 5 steps

```
Setting Input -> Pre-processing Script -> Running Model -> Post-processing Script -> Fetching Output
```

In [7]:
con.tensorset('image', image)
con.scriptexecute('processing_script', 'pre_process', inputs='image', outputs='processed')
con.modelexecute('pytorch_model', 'processed', 'model_out')
con.scriptexecute('processing_script', 'post_process', inputs='model_out', outputs='final')
final = con.tensorget('final')
print(final[0], class_idx[str(final[0])])

281 tabby, tabby catamount


## Running with DAG
Although this looks good, each of these calls has a network overhead of going back and forth and sometimes it's better to run everything as a single execution and that's what you can do with RedisAI DAG. DAGs are much more powerful than that but let's discuss that in another tutorial. Here we first setup a dag object and track all the operations we did above in the dag. Note that none of these tracking steps sends a request to RedisAI server. Once the dag object is ready with all the paths, you can trigger `dag.execute()` to initiate the DAG execution in RedisAI backend

In [8]:
dag = con.dag(routing='default')
dag.tensorset('image', image)
dag.scriptexecute('processing_script', 'pre_process', inputs='image', outputs='processed')
dag.modelexecute('pytorch_model', 'processed', 'model_out')
dag.scriptexecute('processing_script', 'post_process', inputs='model_out', outputs='final')
dag.tensorget('final')

final = dag.execute()[-1]
print(final[0], class_idx[str(final[0])])

281 tabby, tabby catamount
